In [1]:
from utils import SUPPORTED_MODELS
from transformers import AutoConfig
from optimum.intel.openvino import OVModelForCausalLM
from pathlib import Path
import openvino as ov
from transformers import AutoTokenizer

model_id = "red-pajama-3b-chat"
# models_dir = Path("c:/Users/nsavelye/workspace/projects/openvino_notebooks/notebooks/254-llm-chatbot")
models_dir = Path("./")
model_dir = models_dir / model_id / "FP16"
# model_dir = models_dir / model_id / "FP16_calibrated"
# model_dir = models_dir / model_id / "INT8_compressed_weights"
model_configuration = SUPPORTED_MODELS[model_id]

core = ov.Core()
# device = "CPU"
device = "GPU"

tok = AutoTokenizer.from_pretrained(models_dir / model_id / "RedPajama-INCITE-Chat-3B-v1", trust_remote_code=True)

ov_config = {'PERFORMANCE_HINT': 'LATENCY', 'NUM_STREAMS': '1', "CACHE_DIR": ""}
ov_model = OVModelForCausalLM.from_pretrained(model_dir, device=device, ov_config=ov_config,
                                              config=AutoConfig.from_pretrained(model_dir, trust_remote_code=True),
                                              trust_remote_code=True)

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, onnx, openvino


/home/guest/nsavelye/venvs/fp16_calibration/lib/python3.8/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
The argument `trust_remote_code` is to be used along with export=True. It will be ignored.
Compiling the model to GPU ...


In [2]:
import importlib
import shutil
import partially_upcast_nodes_to_fp32
import model_upcast_utils
import main
importlib.reload(partially_upcast_nodes_to_fp32)
importlib.reload(main)


model = partially_upcast_nodes_to_fp32.partially_upcast_nodes_to_fp32(ov_model.model, main.get_inputs_for_calibration(
    ov_model, tok, model_configuration, "<human>: Which lakes are near Munich?\n<bot>:"), batch_size=50, verbose=True)
# model = model_upcast_utils.partially_upcast_nodes_to_fp32(ov_model.model, get_inputs_for_calibration("<human>: Which lakes are near Munich?\n<bot>:"))
calibrated_model_dir = Path(f"{model_dir}_calibrated_custom")
ov.save_model(model, calibrated_model_dir / "openvino_model.xml")
shutil.copy(model_dir / "config.json", calibrated_model_dir / "config.json")
# ov_model = OVModelForCausalLM.from_pretrained(calibrated_model_dir, device=device, ov_config=ov_config,
#                                               config=AutoConfig.from_pretrained(calibrated_model_dir, trust_remote_code=True),
#                                               trust_remote_code=True)
ov_model.model = model
ov_model._original_model = model
ov_model.request = None
ov_model.compile()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
 25%|██▌       | 1/4 [00:53<02:40, 53.35s/it]

Upcasted node __module.model.gpt_neox.layers.1.mlp.dense_4h_to_h/aten::linear/MatMul_543 with 0.10 rel2_diff_ratio 0.050180 and mean_rel_error 0.035856
Upcasted node __module.model.gpt_neox.layers.2.mlp.dense_4h_to_h/aten::linear/MatMul_766 with 0.10 rel2_diff_ratio 0.046124 and mean_rel_error 0.033481
Upcasted node __module.model.gpt_neox.layers.3.mlp.dense_4h_to_h/aten::linear/MatMul_989 with 0.10 rel2_diff_ratio 0.047626 and mean_rel_error 0.033770
Upcasted node __module.model.gpt_neox.layers.4.mlp.dense_4h_to_h/aten::linear/MatMul_1212 with 0.10 rel2_diff_ratio 0.045343 and mean_rel_error 0.033719
Upcasted node __module.model.gpt_neox.layers.5.mlp.dense_4h_to_h/aten::linear/MatMul_1435 with 0.10 rel2_diff_ratio 0.046034 and mean_rel_error 0.033424
Upcasted node __module.model.gpt_neox.layers.6.mlp.dense_4h_to_h/aten::linear/MatMul_1658 with 0.10 rel2_diff_ratio 0.046875 and mean_rel_error 0.033980
Upcasted node __module.model.gpt_neox.layers.7.mlp.dense_4h_to_h/aten::linear/MatMul_

 50%|█████     | 2/4 [01:45<01:45, 52.72s/it]

Upcasted node __module.model.gpt_neox.layers.8.mlp.dense_4h_to_h/aten::linear/MatMul_2104 with 0.10 rel2_diff_ratio 0.048618 and mean_rel_error 0.034572
Upcasted node __module.model.gpt_neox.layers.9.mlp.dense_4h_to_h/aten::linear/MatMul_2327 with 0.10 rel2_diff_ratio 0.046965 and mean_rel_error 0.034050
Upcasted node __module.model.gpt_neox.layers.10.mlp.dense_4h_to_h/aten::linear/MatMul_2550 with 0.10 rel2_diff_ratio 0.047236 and mean_rel_error 0.034274
Upcasted node __module.model.gpt_neox.layers.11.mlp.dense_4h_to_h/aten::linear/MatMul_2773 with 0.10 rel2_diff_ratio 0.045913 and mean_rel_error 0.033175
Upcasted node __module.model.gpt_neox.layers.12.mlp.dense_4h_to_h/aten::linear/MatMul_2996 with 0.10 rel2_diff_ratio 0.046635 and mean_rel_error 0.033646
Upcasted node __module.model.gpt_neox.layers.13.mlp.dense_4h_to_h/aten::linear/MatMul_3219 with 0.10 rel2_diff_ratio 0.047476 and mean_rel_error 0.035168
Upcasted node __module.model.gpt_neox.layers.14.mlp.dense_4h_to_h/aten::linear

 75%|███████▌  | 3/4 [02:38<00:52, 52.87s/it]

Upcasted node __module.model.gpt_neox.layers.16.mlp.dense_4h_to_h/aten::linear/MatMul_3888 with 0.10 rel2_diff_ratio 0.046965 and mean_rel_error 0.033864
Upcasted node __module.model.gpt_neox.layers.17.mlp.dense_4h_to_h/aten::linear/MatMul_4111 with 0.10 rel2_diff_ratio 0.047296 and mean_rel_error 0.034661
Upcasted node __module.model.gpt_neox.layers.18.mlp.dense_4h_to_h/aten::linear/MatMul_4334 with 0.10 rel2_diff_ratio 0.047716 and mean_rel_error 0.034095
Upcasted node __module.model.gpt_neox.layers.19.mlp.dense_4h_to_h/aten::linear/MatMul_4557 with 0.10 rel2_diff_ratio 0.050901 and mean_rel_error 0.036590
Upcasted node __module.model.gpt_neox.layers.20.mlp.dense_4h_to_h/aten::linear/MatMul_4780 with 0.10 rel2_diff_ratio 0.047626 and mean_rel_error 0.034472
Upcasted node __module.model.gpt_neox.layers.21.mlp.dense_4h_to_h/aten::linear/MatMul_5003 with 0.10 rel2_diff_ratio 0.047897 and mean_rel_error 0.035550
Upcasted node __module.model.gpt_neox.layers.22.mlp.dense_4h_to_h/aten::line

100%|██████████| 4/4 [03:24<00:00, 51.11s/it]

Upcasted node __module.model.gpt_neox.layers.25.mlp.dense_4h_to_h/aten::linear/MatMul_5895 with 0.10 rel2_diff_ratio 0.051983 and mean_rel_error 0.036955
Upcasted node __module.model.gpt_neox.layers.26.mlp.dense_4h_to_h/aten::linear/MatMul_6118 with 0.10 rel2_diff_ratio 0.054778 and mean_rel_error 0.038841
Upcasted node __module.model.gpt_neox.layers.27.mlp.dense_4h_to_h/aten::linear/MatMul_6341 with 0.10 rel2_diff_ratio 0.052734 and mean_rel_error 0.035784
Upcasted node __module.model.gpt_neox.layers.28.mlp.dense_4h_to_h/aten::linear/MatMul_6564 with 0.10 rel2_diff_ratio 0.051713 and mean_rel_error 0.036293
Upcasted node __module.model.gpt_neox.layers.29.mlp.dense_4h_to_h/aten::linear/MatMul_6787 with 0.10 rel2_diff_ratio 0.040535 and mean_rel_error 0.030602
Upcasted node __module.model.gpt_neox.layers.30.mlp.dense_4h_to_h/aten::linear/MatMul_7010 with 0.10 rel2_diff_ratio 0.040685 and mean_rel_error 0.031799



Compiling the model to GPU ...


In [3]:
import importlib
import main
importlib.reload(main)

prompt = "Which lakes are near Munich?"

generation_kwargs = dict(
        max_new_tokens=100,
        temperature=0.1,
        do_sample=0.1 > 0.0,
        top_p=1.0,
        top_k=50,
        repetition_penalty=1.2
)


# print(run_generate(ov_model, prompt, model_configuration, **generation_kwargs))
for text in main.run_generate(ov_model, tok, prompt, model_configuration, **generation_kwargs):
    print(text, end="")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/home/guest/nsavelye/venvs/fp16_calibration/lib/python3.8/site-packages/optimum/intel/openvino/modeling_decoder.py:407: FutureWarning: `shared_memory` is deprecated and will be removed in 2024.0. Value of `shared_memory` is going to override `share_inputs` value. Please use only `share_inputs` explicitly.
  self.request.start_async(inputs, shared_memory=True)


Which lakes are near Munich?
<bot>: There is a total of 5 large and 3 small Bavarian Lakes. The largest lake, the Ammersee (German: "Achse" = 'the straight line'), has an area of about 50 km² with over 200 islands in its middle part; it's also one of Germany’s most popular tourist destinations because of its beautiful scenery as well as for water sports like sailing or rowing on Lake Constance which lies to the south-east across the border into